# Cassava Leaf Classification | Artificial Intelligence

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import matplotlib.image as mpimg
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from efficientnet.keras import EfficientNetB3

## Loading Data

In [ ]:
train_dir = 'cassava-leaf-disease-classification/train_images'
test_dir = 'cassava-leaf-disease-classification/test_images' 
train_names = pd.read_csv('cassava-leaf-disease-classification/train.csv')

In [ ]:
train_names['label'] = train_names['label'].apply(str)
train_names.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [ ]:
print(len(train_names))

21397


## Plot the Data

In [ ]:
batch_size = 32
img_height = 600
img_width = 800


## Constructing the train generator

In [ ]:
# Performing on the fly image augmentation
train_datagen = ImageDataGenerator(rescale=1./255.,shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True, validation_split=0.2)

We've applied feature scaling,image augmentation and then split the train set into 2 partitions one for training and the other for validation just by specifying the argument validation_split=0.2 which splits the dataset into to 2 sets where the validation set will have 20% of the total images.

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
dataframe = train_names,
directory = train_dir,
x_col = "image_id",
y_col = "label",
subset = "training",
batch_size = batch_size,
seed = 42,
shuffle = True,
class_mode = "sparse",
target_size = (img_height, img_width))

Found 17118 validated image filenames belonging to 5 classes.


In [ ]:
valid_generator = train_datagen.flow_from_dataframe(
dataframe = train_names,
directory = train_dir,
x_col = "image_id",
y_col = "label",
subset = "validation",
batch_size = batch_size,
seed =42,
shuffle = True,
class_mode ="sparse",
target_size = (img_height,img_width)
)

Found 4279 validated image filenames belonging to 5 classes.


## Iterating Over Data and Plotting it

## Now the data and labels are together!

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(img_height, img_width, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(img_height, img_width, 3))

x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dense(1024,activation="relu")(x)
x = keras.layers.Dense(128,activation="relu")(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5,activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

checkpoint_filepath = 'model/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 600, 800, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 19, 25, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 23,091,501
Trainable params: 2,230,021
Non-trainable params: 20,861,480
_________________________________________

In [ ]:
model.fit(train_generator, epochs=10, validation_data=valid_generator, callbacks=[model_checkpoint_callback])

Epoch 1/10
535/535 [==============================] - 7016s 13s/step - loss: 0.7709 - accuracy: 0.7121 - val_loss: 0.6686 - val_accuracy: 0.7432
Epoch 2/10
535/535 [==============================] - 7008s 13s/step - loss: 0.6662 - accuracy: 0.7545 - val_loss: 0.6461 - val_accuracy: 0.7539
Epoch 3/10
535/535 [==============================] - 7019s 13s/step - loss: 0.6288 - accuracy: 0.7657 - val_loss: 0.6334 - val_accuracy: 0.7595
Epoch 4/10
535/535 [==============================] - 7432s 14s/step - loss: 0.6035 - accuracy: 0.7789 - val_loss: 0.6263 - val_accuracy: 0.7677
Epoch 5/10
535/535 [==============================] - 7854s 15s/step - loss: 0.5783 - accuracy: 0.7886 - val_loss: 0.5850 - val_accuracy: 0.7829
Epoch 6/10
535/535 [==============================] - 7868s 15s/step - loss: 0.5711 - accuracy: 0.7912 - val_loss: 0.5930 - val_accuracy: 0.7806
Epoch 7/10
535/535 [==============================] - 7871s 15s/step - loss: 0.5583 - accuracy: 0.7941 - val_loss: 0.6217 - val_ac